# Analisi strutturale dei datasets

In [1]:
import os
import pandas as pd
import json
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
from itertools import combinations
import shutil
import seaborn as sns

In [2]:
lilla = "#c8a2c8"
color_of_bar1 = "#c8a2c8"
color_of_bar2 = "#6495ED"
color_of_edge = "#000000"

In [3]:
matrix = np.array([[149030, 34],[7442, 12]])
norm = matplotlib.colors.Normalize(matrix.min(), matrix.max())
boundaries = [value for value in matrix.flatten().tolist()]
list.sort(boundaries)
colors = [[norm(boundaries[0]), "#dcc4dc"], 
          [norm(boundaries[1]), "#c8a2c8"], 
          [norm(boundaries[2]), "#93779c"], 
          [norm(boundaries[3]), "#6c4675"]]
my_cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", colors)

In [6]:
dataset_path = "..\\..\\Dataset\\DatasetJSON\\datasets.json"
plot_path = ".\\plot\\"
if os.path.exists(plot_path):
    shutil.rmtree(plot_path)
    os.mkdir(plot_path)
else:
    os.mkdir(plot_path)

### Import dataset JSON ed estrazione numero righe, colonne e celle per ogni dataset

In [7]:
with open(dataset_path, 'r', encoding='utf-8') as f:
    json_object = json.loads(f.read())
    f.close()

In [8]:
# dato un df restituisce numero di righe, colonne e celle
def extract_data_from_df(df):
    return {
        "rows": df.shape[0],
        "columns": df.shape[1],
        "cells": df.shape[0] * df.shape[1]
    }

In [9]:
info_datasets = []
for i in range(0, len(json_object)):
    df = pd.read_json(json_object[i]["dataset"], orient="index", encoding="utf-8")
    info = extract_data_from_df(df)
    info_datasets.append([
        json_object[i]["group_name"],
        json_object[i]["dataset_name"],
        info["rows"],
        info["columns"],
        info["cells"]
    ])

In [10]:
# costruisce un df a partire da un array di array e un numero c che rappresenta la colonna su cui ordinare
# reverse serve a ordinare in senso crescente o descrescente
def get_info_order_by(list_of_info, c, reverse=False):
    sorted_list = sorted(list_of_info, key = lambda x: x[c], reverse=reverse)
    df = pd.DataFrame(sorted_list, columns=["group_name", "dataset_name", "rows", "columns", "cells"])
    return df

In [11]:
df_info = get_info_order_by(info_datasets, 1, True)
display(df_info)

,group_name,dataset_name,rows,columns,cells
0,02-GioPonSpiz,wikipedia,2664,5,13320
1,05-MalPatSaj,wikipedia,3111,7,21777
2,06-MarScoToc,wikipedia,619,5,3095
3,10-DeBiGa,wikipedia,1002,13,13026
4,00-avengers,valuetoday,10682,18,192276
5,02-GioPonSpiz,valuetoday,10680,6,64080
6,03-gren,valuetoday,10682,10,106820
7,04-iGMM,valuetoday,10000,12,120000
8,07-silvestri,valuetoday,1000,9,9000
9,08-slytherin,valuetoday,10000,8,80000


In [12]:
sum(df_info["rows"])

185598

### Plot numero di valori distinti e nulli per colonna, per ogni dataset

In [28]:
unique_null_dict = []
for i in range(0, len(json_object)):
    
    df = pd.read_json(json_object[i]["dataset"], orient="index", encoding="utf-8")
    info = extract_data_from_df(df)

    tmp_unique = {}
    tmp_null = {}
    for key in df:
        try:
            unique_value = len(pd.unique(df[key]))
        except:
            unique_value = len(df[key])
        null_value = df[key].isnull().sum()
        tmp_unique[key] = unique_value
        tmp_null[key] = null_value
            
    unique_null_dict.append({
        "group_name": json_object[i]["group_name"],
        "dataset_name": json_object[i]["dataset_name"],
        "rows": info["rows"],
        "columns": info["columns"],
        "column_name": tmp_unique.keys(),
        "unique_value": tmp_unique,
        "null_value": tmp_null
    })

In [30]:
for item in unique_null_dict:
    item["unique_value"]["ROWS"] = item["rows"]
    item["null_value"]["ROWS"] = item["rows"]
    df = pd.DataFrame([item["unique_value"], item["null_value"]], index=["UNIQUE_VALUE", "NULL_VALUE"])
    temp_cols = df.columns.tolist()
    new_cols = temp_cols[-1:] + temp_cols[:-1]
    df = df[new_cols]
    print(item["group_name"] + "-" + item["dataset_name"])
    display(df)
    print("\n\n")

00-avengers-companiesmarketcap


,ROWS,id,name,rank,market_cap,country,share_price,change_1_day,change_1_year,categories
UNIQUE_VALUE,5897,5897,5896,5886,2131,40,3561,1257,4491,5897
NULL_VALUE,5897,0,0,0,0,156,0,0,249,878





00-avengers-disfold


,ROWS,id,name,official_name,headquarters_country,headquarters_continent,founded,employees,ceo,market_cap,categories,gbp
UNIQUE_VALUE,1000,1000,1000,127,392,89,399,772,794,977,1000,500
NULL_VALUE,1000,0,0,874,93,93,436,124,200,0,0,477





00-avengers-hithorizons


...

,ROWS,name,headquarters,number_of_employees,address,industry,website,market_cap,telephone,revenue
UNIQUE_VALUE,5336,5335,82,4149,5308,30,5327,1347,5184,654
NULL_VALUE,5336,0,0,103,0,0,0,422,94,0





10-DeBiGa-govuk


,ROWS,name,company_number,registered_office_address,company_status,company_type,company_creation_date,nature_of_business
UNIQUE_VALUE,7000,6309,6321,5801,4,5,3423,458
NULL_VALUE,7000,0,0,0,0,0,0,0





10-DeBiGa-wikipedia


,ROWS,name,type,industry,founded,founders,headquarters,key_people,services,revenue,operating_income,total_assets,number_of_employees,website
UNIQUE_VALUE,1002,988,1002,1002,1002,1002,1002,1002,1002,1002,1002,1002,1002,1002
NULL_VALUE,1002,0,0,0,0,0,0,0,0,0,0,0,0,0
